In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle


### Load and preprocess data 

In [31]:
def load_and_preprocess_data(csv_file, pickle_file):
  """
  Loads a CSV file, cleans and preprocesses the data, and stores it in a pickle file.
  Args:
      csv_file (str): Path to the CSV file containing job data.
      pickle_file (str): Path to the pickle file where preprocessed data will be stored.
  Returns:
      pandas.DataFrame: The preprocessed DataFrame containing job data.
  """

  df = pd.read_csv(csv_file)
  df = df.dropna()  # Remove rows with missing values (optional)
  tdif = TfidfVectorizer(stop_words='english')
  df['jobdescription'] = df['jobdescription'].fillna('')
  tdif_matrix = tdif.fit_transform(df['jobdescription'])

  with open(pickle_file, 'wb') as f:
      data = {'df': df, 'tdif': tdif, 'tdif_matrix': tdif_matrix}
      pickle.dump(data, f)

  return df, tdif, tdif_matrix

In [2]:
csv_file = './saved_job_data_2.csv'
pickle_file = 'preprocessed_data.pkl'

try:
  with open(pickle_file, 'rb') as f:
    data = pickle.load(f)
    df = data['df']
    tdif = data['tdif']  # Load TF-IDF vectorizer if needed
    tdif_matrix = data['tdif_matrix']  # Load TF-IDF matrix if needed
    print("Loaded data from pickle file")
except FileNotFoundError:
  print("Pickle file not found, loading and preprocessing data from CSV...")
  df, tdif, tdif_matrix = load_and_preprocess_data(csv_file, pickle_file)

Loaded data from pickle file


In [3]:
tdif_matrix.shape

(3417, 34095)

In [12]:
df.head()

,jobtitle,company,jobdescription,joblocation_address
0,AUTOMATION TEST ENGINEER,"Digital Intelligence Systems, LLC",Looking for Selenium engineers...must have sol...,"Atlanta, GA"
1,Information Security Engineer,University of Chicago/IT Services,The University of Chicago has a rapidly growin...,"Chicago, IL"
2,Business Solutions Architect,"Galaxy Systems, Inc.","GalaxE.SolutionsEvery day, our solutions affec...","Schaumburg, IL"
3,"Java Developer (mid level)- FT- GREAT culture,...",TransTech LLC,Java DeveloperFull-time/direct-hireBolingbrook...,"Bolingbrook, IL"
4,DevOps Engineer,Matrix Resources,Midtown based high tech firm has an immediate ...,"Atlanta, GA"


### Insert to database . job table

In [ ]:
table_columns = ", ".join(df.columns.tolist())
table_columns   

'jobtitle, company, jobdescription, joblocation_address'

In [15]:
sql_insert = f"""INSERT INTO job ({table_columns}) VALUES (%s, %s, %s, ..., %s)"""
sql_insert

'INSERT INTO job (jobtitle, company, jobdescription, joblocation_address) VALUES (%s, %s, %s, ..., %s)'

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21947 entries, 0 to 21999
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   jobtitle             21947 non-null  object
 1   company              21947 non-null  object
 2   jobdescription       21947 non-null  object
 3   joblocation_address  21947 non-null  object
dtypes: object(4)
memory usage: 857.3+ KB


In [20]:
# Accessing data using label (assuming "company" is the column name)
# company_name = df.loc[0, "company"]

# Accessing data using integer position (assuming "company" is at index 1)
company_name = df.iloc[0, 1]  # This would work if "company" is the second column

print(f"Company name at row 1: {company_name}")

Company name at row 1: Digital Intelligence Systems, LLC


### Check recommend

Dung tdif matrix và linear kernel để tính ma trận độ tương đồng cosine

In [4]:
tdif_matrix.shape

(3417, 34095)

In [7]:
# Tính toán ma trận độ tương đồng cosine giữa tất cả các mô tả công việc. Độ tương đồng cosine đo lường mức độ giống nhau của hai tài liệu dựa trên vectơ TF-IDF của chúng.
cosine_sim=linear_kernel(tdif_matrix,tdif_matrix)
type(cosine_sim)

numpy.ndarray

In [12]:
# Tạo một Series ánh xạ các tiêu đề công việc duy nhất với các chỉ mục tương ứng của chúng trong DataFrame (để truy xuất hiệu quả).
indices=pd.Series(df.index, index=df['jobtitle']).drop_duplicates()
type(indices)

pandas.core.series.Series

In [13]:
def get_recommendation(title, cosine_sim=cosine_sim):
    try:
        # Get the index of the input title
        idx = indices[title]

        # Calculate cosine similarity scores for all jobs
        sim_scores = list(enumerate(cosine_sim[idx]))  # Enumerate indices and similarity scores

        # Handle potential empty similarity scores (e.g., no related jobs)
        if not sim_scores:
            print(f"No similar jobs found for '{title}'.")
            return []

        # Sort jobs by similarity score in descending order (most similar first)
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Select the top 15 jobs (excluding the input job at index 0)
        top_15_indices = [i for i, _ in sim_scores[:16]]  # Get top 15 indices (16 to exclude the input)

        # Extract job titles from the DataFrame using the selected indices
        recommendations = df['jobtitle'].iloc[top_15_indices[1:]]  # Exclude input job at index 0

        return recommendations

    except KeyError:
        print(f"Job title '{title}' not found in the data.")
        return []

In [14]:
get_recommendation('Lead DevOps Engineer',cosine_sim)


2900                                 IT Security Engineer
1334                     Multiple Cyber Security Openings
773                         Application Security Engineer
1055                                  IT Security Manager
2134                                   Security Architect
202                             Senior Security Architect
2132                                  IT Security Analyst
2096                                   Security Architect
217     Information Security Manager - IT Security, CI...
1649                             Senior Security Engineer
968             Security Analyst / Security Administrator
2661                     Sr Information Security Engineer
1614                  Sr. Infrastructure Security Analyst
1678                                   AVP Cyber Security
923                                IT Security Consultant
Name: jobtitle, dtype: object

### function insert to database

In [6]:
%pip install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [7]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password=""
)

print(mydb)

In [8]:
import mysql.connector
import pandas as pd



# Get the table columns (assuming you know the column names)
table_columns = ", ".join(df.columns.tolist())  # Convert list to comma-separated string

try:
    cursor = mydb.cursor()

    # Prepare SQL statement with placeholders for data
    sql_insert = f"""INSERT INTO your_table_name ({table_columns}) VALUES (%s, %s, %s, ..., %s)"""
    values = df.to_records(index=False).tolist()  # Convert DataFrame to list of tuples

    cursor.executemany(sql_insert, values)
    connection.commit()

    print(f"Successfully inserted {cursor.rowcount} rows to the database.")

except mysql.connector.Error as err:
    print("Error connecting to database:", err)
finally:
    if connection:
        connection.close()
        cursor.close()


NameError: name 'df' is not defined